In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
louisteitelbaum_911_recordings_path = kagglehub.dataset_download('louisteitelbaum/911-recordings')

print('Data source import complete.')


In [ ]:


import numpy as np # linear algebra
import pandas as pd
#printing the files in input dataset
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/911-recordings/911_recordings/call_71.mp3
/kaggle/input/911-recordings/911_recordings/call_456.mp3
/kaggle/input/911-recordings/911_recordings/call_347.mp3
/kaggle/input/911-recordings/911_recordings/call_664.mp3
/kaggle/input/911-recordings/911_recordings/call_208.mp3
/kaggle/input/911-recordings/911_recordings/call_420.mp3
/kaggle/input/911-recordings/911_recordings/call_589.mp3
/kaggle/input/911-recordings/911_recordings/call_93.mp3
/kaggle/input/911-recordings/911_recordings/call_627.mp3
/kaggle/input/911-recordings/911_recordings/call_648.mp3
/kaggle/input/911-recordings/911_recordings/call_307.mp3
/kaggle/input/911-recordings/911_recordings/call_541.mp3
/kaggle/input/911-recordings/911_recordings/call_336.mp3
/kaggle/input/911-recordings/911_recordings/call_694.mp3
/kaggle/input/911-recordings/911_recordings/call_56.mp3
/kaggle/input/911-recordings/911_recordings/call_557.mp3
/kaggle/input/911-recordings/911_recordings/call_675.mp3
/kaggle/input/911-recordings/911_r

In [ ]:
import numpy as np
import pandas as pd
import os
import librosa
from tqdm import tqdm

In [ ]:
audio_files = [] #list to store extracted features
audio_dir = '/kaggle/input'

In [ ]:
#paths of all audio files
for dirname, _, filenames in os.walk(audio_dir):
    for filename in filenames:
        if filename.endswith('.mp3'):
            audio_files.append(os.path.join(dirname, filename))


In [ ]:
print(f"There are {len(audio_files)} audio files.")


There are 707 audio files.


In [ ]:
#MFCC Feature extractor

def extract_mfcc(file_path, n_mfcc=13):
    try:
        y, sr = librosa.load(file_path, sr=16000)  # Resample to 16kHz
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
        mfcc_mean = np.mean(mfcc.T, axis=0)  # Compute mean across time axis
        return mfcc_mean
    except Exception as e:
        print(f"Could not read {file_path}: {e}")
        return None  # Return None if the file unread

In [ ]:
batch_size = 70
batch_features = []
batch_file_names = []

In [ ]:
for idx, file_path in enumerate(tqdm(audio_files, desc="Processing Audio Files")):
    mfcc_features = extract_mfcc(file_path)

    #features successfully extracted?
    if mfcc_features is not None:
        batch_features.append(mfcc_features)
        batch_file_names.append(file_path)

    #batch = full?  save it : continue
    if len(batch_features) >= batch_size or idx == len(audio_files) - 1:
        #batch to df
        batch_df = pd.DataFrame(batch_features)
        batch_df.columns = [f'mfcc_{i+1}' for i in range(batch_df.shape[1])]
        batch_df['file_name'] = batch_file_names

        #batch to CSV
        batch_df.to_csv("audio_features.csv", mode='a', header=not os.path.exists("audio_features.csv"), index=False)

        #Clear batch lists
        batch_features = []
        batch_file_names = []


Processing Audio Files:   5%|▍         | 33/707 [00:59<15:31,  1.38s/it]Note: Illegal Audio-MPEG-Header 0x7374616e at offset 11139840.
Note: Trying to resync...
Note: Hit end of (available) data during resync.
Processing Audio Files:   6%|▌         | 42/707 [01:07<09:06,  1.22it/s]Note: Illegal Audio-MPEG-Header 0x7374616e at offset 11139840.
Note: Trying to resync...
Note: Hit end of (available) data during resync.
Processing Audio Files:  13%|█▎        | 89/707 [01:55<08:00,  1.29it/s][src/libmpg123/layer3.c:INT123_do_layer3():1841] error: dequantization failed!
[src/libmpg123/layer3.c:INT123_do_layer3():1841] error: dequantization failed!
[src/libmpg123/layer3.c:INT123_do_layer3():1841] error: dequantization failed!
Processing Audio Files:  15%|█▍        | 106/707 [02:17<09:41,  1.03it/s]Note: Illegal Audio-MPEG-Header 0x7374616e at offset 852480.
Note: Trying to resync...
Note: Hit end of (available) data during resync.
Processing Audio Files:  19%|█▉        | 135/707 [02:43<07:57,

Could not read /kaggle/input/911-recordings/911_recordings/call_506.mp3: 


Processing Audio Files:  90%|████████▉ | 636/707 [14:10<01:02,  1.13it/s][src/libmpg123/id3.c:process_extra():681] error: No extra frame text / valid description?
[src/libmpg123/id3.c:process_extra():681] error: No extra frame text / valid description?
Processing Audio Files: 100%|██████████| 707/707 [15:46<00:00,  1.34s/it]


****Feature extraction and saving completed****

wave2vec begin

In [ ]:
!pip install transformers


dwnld wave2vec model locally and upload

In [1]:
from transformers import Wav2Vec2Processor, Wav2Vec2Model

# Download and save the model and processor
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base")

# Save locally to a directory
processor.save_pretrained("wav2vec2-processor")
model.save_pretrained("wav2vec2-model")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/380M [00:00<?, ?B/s]

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
# trying to connect to the net to download Wav2Vec2,
# unable to resolve the
# network address likely due to a lack of internet access in kagg env
import numpy as np
import pandas as pd
import os
import librosa
from tqdm import tqdm
from transformers import Wav2Vec2Processor, Wav2Vec2Model
import torch

#initing Wav2Vec2 processor and model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base")


/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_

In [3]:
# Initialize list to hold file paths of audio files
audio_files = []
audio_dir = '/kaggle/input'

# Collect paths of all audio files
for dirname, _, filenames in os.walk(audio_dir):
    for filename in filenames:
        if filename.endswith('.mp3'):
            audio_files.append(os.path.join(dirname, filename))

print(f"Found {len(audio_files)} audio files.")


Found 0 audio files.
